##Yelp Restaurant Reviews Classification

This Yelp dataset has information on restaurants (e.g., type of food, price range, etc.) as well as reviews written by patrons. The output variable is the star rating (1-5). We have converted this rating to high (ratings of 4 & 5) and low (1, 2, 3). 

In [35]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline
import statsmodels.api as sm
from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import neighbors
from sklearn import metrics
from sklearn import cross_validation
from sklearn import naive_bayes
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import tree

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
import pylab
import re
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from operator import itemgetter
from sklearn.metrics import classification_report
import csv
import os
from sklearn.feature_extraction.text import CountVectorizer


from datetime import datetime


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [36]:
reviews = pd.read_csv("Yelp.csv", header=0)
review_names = reviews.columns.tolist()
reviews_numeric = reviews[review_names[0:20]]

In [ ]:
reviews['target'] = np.where(reviews['stars'] > 3, 1, 0)
reviews_numeric['target'] = np.where(reviews_numeric['stars'] > 3, 1, 0)

In [38]:
reviews[['Cheap', 'Moderate', 'Expensive', 'VeryExpensive', 'American', 'Chinese', 'French', 'Japanese', 'Indian', 'Italian', 'Greek', 'Mediterranean', 'Mexican', 'Thai', 'Vietnamese', 'Others']] = reviews[['Cheap', 'Moderate', 'Expensive', 'VeryExpensive', 'American', 'Chinese', 'French', 'Japanese', 'Indian', 'Italian', 'Greek', 'Mediterranean', 'Mexican', 'Thai', 'Vietnamese', 'Others']].astype(bool)
reviews.dtypes

stars             int64
votes_cool        int64
votes_funny       int64
votes_useful      int64
Cheap              bool
Moderate           bool
Expensive          bool
VeryExpensive      bool
American           bool
Chinese            bool
French             bool
Japanese           bool
Indian             bool
Italian            bool
Greek              bool
Mediterranean      bool
Mexican            bool
Thai               bool
Vietnamese         bool
Others             bool
Review           object
target            int32
dtype: object

In [30]:
reviews.describe()

#y, X = dmatrices('stars ~ votes_cool + votes_funny + votes_useful + Cheap + Moderate + Expensive + VeryExpensive + American + Chinese + French + Japanese + Indian + Italian + Greek + Mediterranean + Mexican + Thai + Vietnamese + Others', data=reviews_numeric, return_type='dataframe')
#model = sm.OLS(y, X)
#result = model.fit()
#print result.summary()

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Japanese,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,target
count,19999.000000,19999.000000,19999.000000,19999.000000,19999,19999,19999,19999,19999,19999,...,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999
mean,3.773289,0.917746,0.689484,1.465273,0.300465,0.6185809,0.06435322,0.01660083,0.3033152,0.1092055,...,0.05505275,0.01430072,0.2112606,0.02090105,0.06355318,0.2586629,0.01150058,0.01265063,0.07625381,0.6776839
std,1.153749,1.970018,1.801502,2.275016,0.4584717,0.4857472,0.2453872,0.1277735,0.4597017,0.3119046,...,0.2280889,0.1187304,0.408213,0.1430567,0.2439614,0.437911,0.106625,0.1117641,0.2654104,0.467375
min,1.000000,0.000000,0.000000,0.000000,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25%,3.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50%,4.000000,0.000000,0.000000,1.000000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
75%,5.000000,1.000000,1.000000,2.000000,1,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
max,5.000000,66.000000,57.000000,68.000000,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [53]:
reviews.describe()

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,French,Japanese,Indian,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others
count,19999.000000,19999.000000,19999.000000,19999.000000,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999,19999
mean,3.773289,0.917746,0.689484,1.465273,0.300465,0.6185809,0.06435322,0.01660083,0.3033152,0.1092055,0.01630082,0.05505275,0.01430072,0.2112606,0.02090105,0.06355318,0.2586629,0.01150058,0.01265063,0.07625381
std,1.153749,1.970018,1.801502,2.275016,0.4584717,0.4857472,0.2453872,0.1277735,0.4597017,0.3119046,0.1266329,0.2280889,0.1187304,0.408213,0.1430567,0.2439614,0.437911,0.106625,0.1117641,0.2654104
min,1.000000,0.000000,0.000000,0.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
25%,3.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50%,4.000000,0.000000,0.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
75%,5.000000,1.000000,1.000000,2.000000,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
max,5.000000,66.000000,57.000000,68.000000,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [39]:
B, A = dmatrices('target ~ votes_cool + votes_funny + votes_useful + Cheap + Moderate + Expensive + VeryExpensive + American + Chinese + French + Japanese + Indian + Italian + Greek + Mediterranean + Mexican + Thai + Vietnamese + Others', data=reviews, return_type='dataframe')
b = B['target'].values
B[:10]

,target
0,0
1,0
2,1
3,0
4,0
5,0
6,1
7,0
8,1
9,0


Ignore the text (reviews) and run a classification model with the numeric data (you can use standard methods like logistic regression, k-nearest neighbors or anything else).

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.cross_validation import train_test_split
from sklearn import tree

model_list = [('Decision Tree', tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)),
              ('Gradient Boosting', GradientBoostingClassifier(n_estimators=200, max_depth=4, random_state=1)),          
              ('Random Forest', RandomForestClassifier(n_estimators=10, max_features = "auto", max_depth = 4, random_state=1)),    
             ('Bagging', BaggingClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth = 4), random_state=1))]

In [8]:
A_train, A_test, b_train, b_test = train_test_split(A, b, test_size=0.25, random_state=1)

In [9]:
for (name, model) in model_list:     
    print 'Fitting', name 
    model.fit(A_train, b_train)

Fitting Decision Tree
Fitting Gradient Boosting
Fitting Random Forest
Fitting Bagging


In [10]:
from sklearn import metrics
accuracy_train = {} 
accuracy_test = {}
for (name, model) in model_list:     
    prediction_train = model.predict(A_train)     
    accuracy_train[name] = metrics.accuracy_score(b_train, prediction_train)
    prediction_test = model.predict(A_test)     
    accuracy_test[name] = metrics.accuracy_score(b_test, prediction_test)     
    
df = DataFrame({'Training Accuracy':accuracy_train, 'Test Accuracy':accuracy_test}) 
df

,Test Accuracy,Training Accuracy
Bagging,0.6844,0.686846
Decision Tree,0.6862,0.684912
Gradient Boosting,0.6902,0.701913
Random Forest,0.6766,0.678045


Perform a supervised classification on a subset of the corpus using the reviews only.

In [11]:
reviews_random = reviews.reindex(np.random.permutation(reviews.index))

In [12]:
def mhttp://localhost:8888/notebooks/Group%20Homework%202.ipynb#Bake_all_lower_letters(string):
    letters_only = re.sub('[^a-zA-Z]', ' ', string)
    letters_only_lowered = letters_only.lower()
    return letters_only_lowered

In [13]:
reviews_random['Review'] = reviews_random['Review'].map(make_all_lower_letters)

In [14]:
X_train = reviews_random['Review'][:15000]
X_test = reviews_random['Review'][15000:]
Y_train = reviews_random['target'][:15000]
Y_test = reviews_random['target'][15000:]

In [15]:
vectorizer = CountVectorizer()
dtm_train = vectorizer.fit_transform(X_train)
dtm_test = vectorizer.transform(X_test)

In [16]:
nb_classifier = MultinomialNB().fit(dtm_train, Y_train)

In [17]:
y_nb_predicted = nb_classifier.predict(dtm_test)

In [18]:
metrics.confusion_matrix(Y_test, y_nb_predicted)

array([[1135,  486],
       [ 270, 3108]])

In [19]:
metrics.accuracy_score(Y_test, y_nb_predicted)

0.84876975395079013

In [20]:
y_nb_actual = nb_classifier.predict(dtm_train)
y_nb = list(y_nb_actual) + list(y_nb_predicted)
type(y_nb)

list

Combine the numeric data and the text classification model to create a “hybrid” model.

In [21]:
reviews_random['TextPredictScore'] = y_nb
reviews_random[:10]

,stars,votes_cool,votes_funny,votes_useful,Cheap,Moderate,Expensive,VeryExpensive,American,Chinese,...,Italian,Greek,Mediterranean,Mexican,Thai,Vietnamese,Others,Review,target,TextPredictScore
6751,5,1,0,2,False,True,False,False,False,False,...,True,False,False,False,False,False,False,my first experience lgo was w much in trepid...,True,True
584,1,0,2,2,True,False,False,False,False,False,...,True,False,False,False,False,False,False,round table pizza is usually the bomb in this...,False,False
3111,4,3,1,1,False,True,False,False,False,False,...,False,False,False,True,False,False,False,this has become my standby for mexican food in...,True,True
19997,3,0,1,0,False,False,True,False,False,False,...,False,False,False,False,False,False,True,the location is convenient and i would say tha...,False,False
10533,5,2,1,2,False,True,False,False,False,True,...,False,False,False,False,False,False,False,a good family restaurant authentic hong kong ...,True,True
15798,2,1,1,1,True,False,False,False,False,False,...,False,True,True,False,False,False,False,i am really surprised at all the good reviews ...,False,False
1046,1,0,1,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,very slow service,False,True
11207,2,0,0,0,False,False,True,False,False,True,...,False,False,False,False,False,False,False,apparently i am not getting this place the se...,False,False
8244,2,3,3,2,False,True,False,False,True,False,...,False,False,False,False,False,False,False,restauranting is not for the faint of heart i...,False,False
3181,4,0,0,1,False,True,False,False,False,False,...,False,False,False,True,False,False,False,awesome new mexican food done right great at...,True,True


In [ ]:
D, C = dmatrices('target ~ votes_cool + votes_funny + votes_useful + Cheap + Moderate + Expensive + VeryExpensive + American + Chinese + French + Japanese + Indian + Italian + Greek + Mediterranean + Mexican + Thai + Vietnamese + Others + TextPredictScore', data=reviews_random, return_type='dataframe')
d = D['target'].values

In [33]:
model_list = [('Decision Tree', tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)),
              ('Gradient Boosting', GradientBoostingClassifier(n_estimators=200, max_depth=4, random_state=1)),          
              ('Random Forest', RandomForestClassifier(n_estimators=10, max_features = "auto", max_depth = 4, random_state=1)),    
             ('Bagging', BaggingClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth = 4), random_state=1))]

In [34]:
C_train, C_test, d_train, d_test = train_test_split(C, d, test_size=0.25, random_state=1)

In [35]:
for (name, model) in model_list:     
    print 'Fitting', name 
    model.fit(C_train, d_train)

Fitting Decision Tree
Fitting Gradient Boosting
Fitting Random Forest
Fitting Bagging


In [36]:
from sklearn import metrics
accuracy_train1 = {} 
accuracy_test1 = {}
for (name, model) in model_list:     
    prediction_train1 = model.predict(C_train)     
    accuracy_train1[name] = metrics.accuracy_score(d_train, prediction_train1)
    prediction_test1 = model.predict(C_test)     
    accuracy_test1[name] = metrics.accuracy_score(d_test, prediction_test1)     
    
df1 = DataFrame({'Training Accuracy':accuracy_train1, 'Test Accuracy':accuracy_test1}) 
df1

,Test Accuracy,Training Accuracy
Bagging,0.8850,0.883659
Decision Tree,0.8848,0.883992
Gradient Boosting,0.8836,0.887192
Random Forest,0.8834,0.881992


In [37]:
model2 = tree.DecisionTreeClassifier(criterion='entropy', max_depth = 4)
result2 = model2.fit(C_train, d_train)

prediction_train2 = model2.predict(C_train)
print metrics.accuracy_score(d_train, prediction_train2)
prediction2 = model2.predict(C_test)
print metrics.accuracy_score(d_test, prediction2)


0.883992266151
0.8848


In [38]:
from StringIO import StringIO
import pydot
dot_data = StringIO()
tree.export_graphviz(model2, out_file=dot_data, feature_names=C.columns.values)
pydot.graph_from_dot_data(dot_data.getvalue()).write_png('Reviews.png')

True

Use unsupervised sentiment analysis on the reviews and use the sentiment score to predict high/low rating. 

In [39]:
sentiment_results = pd.read_csv("YelpResults.csv", header=0)

In [40]:
sentiment_list = [0]
scores = list(sentiment_results["Score"])
sentiment_list = sentiment_list + scores
reviews["Score"] = sentiment_list

Sentiment only

In [41]:
N, M = dmatrices('target ~ Score', data=reviews, return_type='dataframe')
n = N['target'].values

In [42]:
model_list = [('Decision Tree', tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)),
              ('Gradient Boosting', GradientBoostingClassifier(n_estimators=200, max_depth=4, random_state=1)),          
              ('Random Forest', RandomForestClassifier(n_estimators=10, max_features = "auto", max_depth = 4, random_state=1)),    
             ('Bagging', BaggingClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth = 4), random_state=1))]

In [43]:
M_train, M_test, n_train, n_test = train_test_split(M, n, test_size=0.25, random_state=1)

In [44]:
for (name, model) in model_list:     
    print 'Fitting', name 
    model.fit(M_train, n_train)

Fitting Decision Tree
Fitting Gradient Boosting
Fitting Random Forest
Fitting Bagging


In [45]:
from sklearn import metrics
accuracy_train3 = {} 
accuracy_test3 = {}
for (name, model) in model_list:     
    prediction_train3 = model.predict(M_train)     
    accuracy_train3[name] = metrics.accuracy_score(n_train, prediction_train3)
    prediction_test3 = model.predict(M_test)     
    accuracy_test3[name] = metrics.accuracy_score(n_test, prediction_test3)     
    
df = DataFrame({'Training Accuracy':accuracy_train3, 'Test Accuracy':accuracy_test3}) 
df

,Test Accuracy,Training Accuracy
Bagging,0.7162,0.727115
Decision Tree,0.7162,0.727115
Gradient Boosting,0.7162,0.727115
Random Forest,0.7162,0.727115


Sentiment only - pivot table of sentiment against stars

In [47]:
pd.crosstab(index=reviews['target'], columns=reviews['Score'])

Score,-4,-3,-2,-1,0,1,2,3,4
target,,,,,,,,,
0,5,92,428,1299,1823,1887,795,116,1
1,0,8,95,787,2031,4353,4845,1366,68


Model with sentiment and non-review variables

In [48]:
K, J = dmatrices('target ~ votes_cool + votes_funny + votes_useful + Cheap + Moderate + Expensive + VeryExpensive + American + Chinese + French + Japanese + Indian + Italian + Greek + Mediterranean + Mexican + Thai + Vietnamese + Others + Score', data=reviews, return_type='dataframe')
k = K['target'].values

In [49]:
model_list = [('Decision Tree', tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)),
              ('Gradient Boosting', GradientBoostingClassifier(n_estimators=200, max_depth=4, random_state=1)),          
              ('Random Forest', RandomForestClassifier(n_estimators=10, max_features = "auto", max_depth = 4, random_state=1)),    
             ('Bagging', BaggingClassifier(tree.DecisionTreeClassifier(criterion='entropy', max_depth = 4), random_state=1))]

In [50]:
J_train, J_test, k_train, k_test = train_test_split(M, n, test_size=0.25, random_state=1)

In [51]:
for (name, model) in model_list:     
    print 'Fitting', name 
    model.fit(J_train, k_train)

Fitting Decision Tree
Fitting Gradient Boosting
Fitting Random Forest
Fitting Bagging


In [52]:
from sklearn import metrics
accuracy_train4 = {} 
accuracy_test4 = {}
for (name, model) in model_list:     
    prediction_train4 = model.predict(J_train)     
    accuracy_train4[name] = metrics.accuracy_score(k_train, prediction_train4)
    prediction_test4 = model.predict(J_test)     
    accuracy_test4[name] = metrics.accuracy_score(k_test, prediction_test4)     
    
df4 = DataFrame({'Training Accuracy':accuracy_train4, 'Test Accuracy':accuracy_test4}) 
df4

,Test Accuracy,Training Accuracy
Bagging,0.7162,0.727115
Decision Tree,0.7162,0.727115
Gradient Boosting,0.7162,0.727115
Random Forest,0.7162,0.727115


Use unsupervised clustering on the text.

In [53]:
vectorizer = TfidfVectorizer()
dtm_train1 = vectorizer.fit_transform(reviews_random["Review"])
#dtm_test1 = vectorizer.transform(X_test)

In [54]:
from sklearn.cluster import KMeans, MiniBatchKMeans

In [56]:
kmeans_classifier = KMeans(n_clusters=2).fit(dtm_train1, Y_train)

In [57]:
reviews_random['cluster'] = kmeans_classifier.labels_

In [58]:
pd.crosstab(index=reviews_random['target'], columns=reviews_random['cluster'])

cluster,0,1
target,,
0,2878,3568
1,8341,5212
